#임베딩 벡터 사용 방법 1: 임베딩 테이블 (Layer) 만들기

nn.Embedding()을 톹해 구현

#임베딩 Layer = 룩업 테이블

Word Embedding 과정

단어 -> 고유한 정수 부여 -> 임베딩 layer와 내적 -> 임베딩 벡터

임베딩 layer의 각 행은 vocab에 있는 단어 하나와 대응한다. 임베딩 layer의 row = vocab 길이, column = embedding vector의 차원




#파이썬으로만 구현


In [4]:
train_data = "you need to know how to code"

#vocab 만들기
word_set = set(train_data.split())

#정수 mapping
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'you': 2, 'how': 3, 'to': 4, 'know': 5, 'need': 6, 'code': 7, '<unk>': 0, '<pad>': 1}


In [5]:
import torch

In [6]:
#임베딩 벡터의 차원을 3, vocab 길이만큼 행 설정
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [7]:
sample = 'you need to run'.split()
indexes = []

for word in sample:
  try:
    indexes.append(vocab[word])
  except KeyError:
    indexes.append(vocab['<unk>'])
indexes = torch.LongTensor(indexes)

lookup_table = embedding_table[indexes, :]
print(lookup_table)


tensor([[0.2000, 0.9000, 0.3000],
        [0.1000, 0.8000, 0.9000],
        [0.2000, 0.1000, 0.8000],
        [0.0000, 0.0000, 0.0000]])


#nn.Embedding 사용하기

In [8]:
train_data = 'you need to know how to code'

word_set = set(train_data.split())

vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=3, padding_idx=1)


In [9]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-1.1858, -0.3080, -0.3831],
        [ 0.0000,  0.0000,  0.0000],
        [-1.2776,  0.3546, -0.1265],
        [ 0.1807, -0.4749,  0.9652],
        [-0.9273, -0.3804,  1.8888],
        [-0.0056, -0.5643, -0.2401],
        [ 1.6561, -1.2417, -0.5832],
        [ 0.6983,  0.5729,  0.3822]], requires_grad=True)


#임베딩 벡터 사용 방법 2: Pre Trained Word Embedding

학습 데이터가 많지 않다면, 직접 layer를 구하는 것보다 이미 학습된 임베딩 벡터들을 사용하는 것이 낫다.

직접 모델을 구현하고 사전 훈련된 워드 임베딩과 구현해보자

In [10]:
!pip install gensim

In [11]:
import numpy as np
from collections import Counter
import gensim

문장의 긍부적을 판단하는 감성 분류 모델: Positive: 1 Negative: 0


In [12]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]


In [13]:
tokenized_sentences = [line.split() for line in sentences]
print("단어 토큰화 결과: ", tokenized_sentences)


단어 토큰화 결과:  [['nice', 'great', 'best', 'amazing'], ['stop', 'lies'], ['pitiful', 'nerd'], ['excellent', 'work'], ['supreme', 'quality'], ['bad'], ['highly', 'respectable']]


단어 집합 만들어보자

In [14]:
vocab = []
for line in tokenized_sentences:
  for word in line:
    vocab.append(word)

word_counts = Counter(vocab)
print("총 단어 수: ",len(word_counts))

총 단어 수:  15


단어들을 등장 빈도가 높은 순서부터 정렬

In [15]:
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab)

['nice', 'great', 'best', 'amazing', 'stop', 'lies', 'pitiful', 'nerd', 'excellent', 'work', 'supreme', 'quality', 'bad', 'highly', 'respectable']


In [16]:
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab):
  word_to_index[word] = index+2

vocab_size = len(word_to_index)
print("패딩 토큰, UNK 토큰을 고려한 단어 집합의 크기: ", vocab_size)

패딩 토큰, UNK 토큰을 고려한 단어 집합의 크기:  17


In [17]:
print(word_to_index)

{'<PAD>': 0, '<UNK>': 1, 'nice': 2, 'great': 3, 'best': 4, 'amazing': 5, 'stop': 6, 'lies': 7, 'pitiful': 8, 'nerd': 9, 'excellent': 10, 'work': 11, 'supreme': 12, 'quality': 13, 'bad': 14, 'highly': 15, 'respectable': 16}


In [18]:
def text_to_sequences(train_data, word_to_index):
  encoded_train_data = []
  for line in train_data:
    index_sequences = []
    for word in line:
      try:
        index_sequences.append(word_to_index[word])
      except KeyError:
        index_sequences.append(word_to_index['<UNK>'])
    encoded_train_data.append(index_sequences)
  return encoded_train_data

train_data_encoded = text_to_sequences(tokenized_sentences, word_to_index)
print(train_data_encoded)

[[2, 3, 4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14], [15, 16]]


최대 길이를 base로 padding하기

In [19]:
max_len = max(len(l) for l in train_data_encoded)
print(max_len)

4


임베딩 벡터 패딩하여 임베딩 벡터 최종 결과 구하자.

In [20]:
def pad_sequences(sentences, max_len):
  #각 문장에 대해 max_len만큼의 0으로 이루어진 numpy array 형성
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    #sentence 존재한다면,
    if len(sentence) != 0:
      #그 sentence에 대응하는 numpy line에 sentence값을 넣는다
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

X_train = pad_sequences(train_data_encoded, max_len=max_len)
print("패딩 결과: ")
print(X_train)

패딩 결과: 
[[ 2  3  4  5]
 [ 6  7  0  0]
 [ 8  9  0  0]
 [10 11  0  0]
 [12 13  0  0]
 [14  0  0  0]
 [15 16  0  0]]


임베딩 벡터를 이용하여 감정 분류하는 Binary Classifier simple model 만들기


In [21]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

class SimpleModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(SimpleModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(embedding_dim * max_len, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    embedded = self.embedding(x)

    flattened = self.flatten(embedded)

    output = self.fc(flattened)
    return self.sigmoid(output)


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_dim = 100
simple_model = SimpleModel(vocab_size, embedding_dim).to(device)


In [23]:
criterion = nn.BCELoss()
optimizer = Adam(simple_model.parameters())


In [24]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [25]:
print(len(train_dataloader))

4


In [26]:
for epoch in range(10):
  for inputs, targets in train_dataloader:
    inputs, targets = inputs.to(device), targets.to(device)

    optimizer.zero_grad()

    outputs = simple_model(inputs).view(-1)

    loss = criterion(outputs, targets)
    loss.backward()

    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.4546612501144409
Epoch 2, Loss: 0.4116010069847107
Epoch 3, Loss: 0.3368053734302521
Epoch 4, Loss: 0.26792916655540466
Epoch 5, Loss: 0.21520820260047913
Epoch 6, Loss: 0.17791545391082764
Epoch 7, Loss: 0.15205876529216766
Epoch 8, Loss: 0.1336413472890854
Epoch 9, Loss: 0.11963801831007004
Epoch 10, Loss: 0.10808978229761124


사전 훈련된 임베딩을 사용하자

밑에는 구글에서 사전 학습시킨 Word2Vec 모델이다

In [34]:
!pip install gdown


In [36]:
# 구글의 사전 훈련된 Word2vec 모델을 로드합니다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Pytorch_Study/자연어처리pytorch/GoogleNews-vectors-negative300.bin.gz', binary=True)
#구글의 Word2Vec은 embedding vector를 300차원으로 구성하였다.

비어있는 행렬을 만들자. 이걸로 google word2vec의 embedding vector를 알아내보자. vocab에 존재하는 각 단어의 embedding vector를 저장할 것이다.

In [37]:
embedding_matrix = np.zeros((vocab_size, 300))
print("임베딩 행렬의 크기: ", embedding_matrix.shape)

임베딩 행렬의 크기:  (17, 300)


word2vec_model은 특정 단어를 입력하면 해당 단어의 임베딩 벡터를 리턴받을텐데, 이 모델에 특정 단어가 존재하지 않는다면, none을 리턴하도록 한다.

In [38]:
def get_vector(word):
  if word in word2vec_model:
    return word2vec_model[word]
  else:
    return None

In [39]:
for word, i in word_to_index.items():
  if i>2:
    temp = get_vector(word)
    if temp is not None:
      embedding_matrix[i] = temp

In [40]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.06298828,  0.12451172,  0.11328125, ..., -0.06347656,
         0.11474609,  0.03100586],
       [ 0.05078125, -0.22753906, -0.13085938, ..., -0.05664062,
         0.10253906,  0.07470703],
       [ 0.15039062,  0.28515625, -0.02319336, ..., -0.20019531,
        -0.11230469,  0.10302734]])

In [41]:
word_to_index['great']

3

In [42]:
np.all(word2vec_model['great'] == embedding_matrix[3])

True

이 사전 훈련 모델을 이용한 Binary Classifier를 구성해보자

In [46]:
class PretrainedEmbeddingModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(PretrainedEmbeddingModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
    self.embedding.weight.requires_grad = True
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(embedding_dim*max_len, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    embedded = self.embedding(x)
    flattened = self.flatten(embedded)
    output = self.fc(flattened)
    return self.sigmoid(output)



In [47]:
pretrained_embedding_model = PretrainedEmbeddingModel(vocab_size, 300).to(device)

In [49]:
criterion = nn.BCELoss()
optimizer = Adam(pretrained_embedding_model.parameters())


In [51]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [52]:
for epoch in range(10):
  for inputs, targets in train_dataloader:
    inputs, targets = inputs.to(device), targets.to(device)

    optimizer.zero_grad()

    outputs = pretrained_embedding_model(inputs).view(-1)

    loss = criterion(outputs, targets)
    loss.backward()

    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.7178673148155212
Epoch 2, Loss: 0.6515899896621704
Epoch 3, Loss: 0.5859054923057556
Epoch 4, Loss: 0.5242006778717041
Epoch 5, Loss: 0.46734774112701416
Epoch 6, Loss: 0.41553616523742676
Epoch 7, Loss: 0.3686983585357666
Epoch 8, Loss: 0.3266470730304718
Epoch 9, Loss: 0.28912681341171265
Epoch 10, Loss: 0.25583571195602417
